# Forward Propagation, Backward Propagation, and Computational Graphs
:label:`sec_backprop`

So far, we have trained our models
with minibatch stochastic gradient descent.
However, when we implemented the algorithm,
we only worried about the calculations involved
in *forward propagation* through the model.
When it came time to calculate the gradients,
we just invoked the backpropagation function provided by the deep learning framework.

The automatic calculation of gradients
profoundly simplifies
the implementation of deep learning algorithms.
Before automatic differentiation,
even small changes to complicated models required
recalculating complicated derivatives by hand.
Surprisingly often, academic papers had to allocate
numerous pages to deriving update rules.
While we must continue to rely on automatic differentiation
so we can focus on the interesting parts,
you ought to know how these gradients
are calculated under the hood
if you want to go beyond a shallow
understanding of deep learning.

In this section, we take a deep dive
into the details of *backward propagation*
(more commonly called *backpropagation*).
To convey some insight for both the
techniques and their implementations,
we rely on some basic mathematics and computational graphs.
To start, we focus our exposition on
a one-hidden-layer MLP
with weight decay ($\ell_2$ regularization, to be described in subsequent chapters).

## Forward Propagation

*Forward propagation* (or *forward pass*) refers to the calculation and storage
of intermediate variables (including outputs)
for a neural network in order
from the input layer to the output layer.
We now work step-by-step through the mechanics
of a neural network with one hidden layer.
This may seem tedious but in the eternal words
of funk virtuoso James Brown,
you must "pay the cost to be the boss".


For the sake of simplicity, let's assume
that the input example is $\mathbf{x}\in \mathbb{R}^d$
and that our hidden layer does not include a bias term.
Here the intermediate variable is:

$$\mathbf{z}= \mathbf{W}^{(1)} \mathbf{x},$$

where $\mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$
is the weight parameter of the hidden layer.
After running the intermediate variable
$\mathbf{z}\in \mathbb{R}^h$ through the
activation function $\phi$
we obtain our hidden activation vector of length $h$:

$$\mathbf{h}= \phi (\mathbf{z}).$$

The hidden layer output $\mathbf{h}$
is also an intermediate variable.
Assuming that the parameters of the output layer
possess only a weight of
$\mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$,
we can obtain an output layer variable
with a vector of length $q$:

$$\mathbf{o}= \mathbf{W}^{(2)} \mathbf{h}.$$

Assuming that the loss function is $l$
and the example label is $y$,
we can then calculate the loss term
for a single data example,

$$L = l(\mathbf{o}, y).$$

As we will see the definition of $\ell_2$ regularization
to be introduced later,
given the hyperparameter $\lambda$,
the regularization term is

$$s = \frac{\lambda}{2} \left(\|\mathbf{W}^{(1)}\|_\textrm{F}^2 + \|\mathbf{W}^{(2)}\|_\textrm{F}^2\right),$$
:eqlabel:`eq_forward-s`

where the Frobenius norm of the matrix
is simply the $\ell_2$ norm applied
after flattening the matrix into a vector.
Finally, the model's regularized loss
on a given data example is:

$$J = L + s.$$

We refer to $J$ as the *objective function*
in the following discussion.


## Computational Graph of Forward Propagation

Plotting *computational graphs* helps us visualize
the dependencies of operators
and variables within the calculation.
:numref:`fig_forward` contains the graph associated
with the simple network described above,
where squares denote variables and circles denote operators.
The lower-left corner signifies the input
and the upper-right corner is the output.
Notice that the directions of the arrows
(which illustrate data flow)
are primarily rightward and upward.

![Computational graph of forward propagation.](http://d2l.ai/_images/forward.svg)
:label:`fig_forward`

## Backpropagation

*Backpropagation* refers to the method of calculating
the gradient of neural network parameters.
In short, the method traverses the network in reverse order,
from the output to the input layer,
according to the *chain rule* from calculus.
The algorithm stores any intermediate variables
(partial derivatives)
required while calculating the gradient
with respect to some parameters.
Assume that we have functions
$\mathsf{Y}=f(\mathsf{X})$
and $\mathsf{Z}=g(\mathsf{Y})$,
in which the input and the output
$\mathsf{X}, \mathsf{Y}, \mathsf{Z}$
are tensors of arbitrary shapes.
By using the chain rule,
we can compute the derivative
of $\mathsf{Z}$ with respect to $\mathsf{X}$ via

$$\frac{\partial \mathsf{Z}}{\partial \mathsf{X}} = \textrm{prod}\left(\frac{\partial \mathsf{Z}}{\partial \mathsf{Y}}, \frac{\partial \mathsf{Y}}{\partial \mathsf{X}}\right).$$

Here we use the $\textrm{prod}$ operator
to multiply its arguments
after the necessary operations,
such as transposition and swapping input positions,
have been carried out.
For vectors, this is straightforward:
it is simply matrix--matrix multiplication.
For higher dimensional tensors,
we use the appropriate counterpart.
The operator $\textrm{prod}$ hides all the notational overhead.

Recall that
the parameters of the simple network with one hidden layer,
whose computational graph is in :numref:`fig_forward`,
are $\mathbf{W}^{(1)}$ and $\mathbf{W}^{(2)}$.
The objective of backpropagation is to
calculate the gradients $\partial J/\partial \mathbf{W}^{(1)}$
and $\partial J/\partial \mathbf{W}^{(2)}$.
To accomplish this, we apply the chain rule
and calculate, in turn, the gradient of
each intermediate variable and parameter.
The order of calculations are reversed
relative to those performed in forward propagation,
since we need to start with the outcome of the computational graph
and work our way towards the parameters.
The first step is to calculate the gradients
of the objective function $J=L+s$
with respect to the loss term $L$
and the regularization term $s$:

$$\frac{\partial J}{\partial L} = 1 \; \textrm{and} \; \frac{\partial J}{\partial s} = 1.$$

Next, we compute the gradient of the objective function
with respect to variable of the output layer $\mathbf{o}$
according to the chain rule:

$$
\frac{\partial J}{\partial \mathbf{o}}
= \textrm{prod}\left(\frac{\partial J}{\partial L}, \frac{\partial L}{\partial \mathbf{o}}\right)
= \frac{\partial L}{\partial \mathbf{o}}
\in \mathbb{R}^q.
$$

Next, we calculate the gradients
of the regularization term
with respect to both parameters:

$$\frac{\partial s}{\partial \mathbf{W}^{(1)}} = \lambda \mathbf{W}^{(1)}
\; \textrm{and} \;
\frac{\partial s}{\partial \mathbf{W}^{(2)}} = \lambda \mathbf{W}^{(2)}.$$

Now we are able to calculate the gradient
$\partial J/\partial \mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$
of the model parameters closest to the output layer.
Using the chain rule yields:

$$\frac{\partial J}{\partial \mathbf{W}^{(2)}}= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \mathbf{o}}{\partial \mathbf{W}^{(2)}}\right) + \textrm{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf{W}^{(2)}}\right)= \frac{\partial J}{\partial \mathbf{o}} \mathbf{h}^\top + \lambda \mathbf{W}^{(2)}.$$
:eqlabel:`eq_backprop-J-h`

To obtain the gradient with respect to $\mathbf{W}^{(1)}$
we need to continue backpropagation
along the output layer to the hidden layer.
The gradient with respect to the hidden layer output
$\partial J/\partial \mathbf{h} \in \mathbb{R}^h$ is given by


$$
\frac{\partial J}{\partial \mathbf{h}}
= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \mathbf{o}}{\partial \mathbf{h}}\right)
= {\mathbf{W}^{(2)}}^\top \frac{\partial J}{\partial \mathbf{o}}.
$$

Since the activation function $\phi$ applies elementwise,
calculating the gradient $\partial J/\partial \mathbf{z} \in \mathbb{R}^h$
of the intermediate variable $\mathbf{z}$
requires that we use the elementwise multiplication operator,
which we denote by $\odot$:

$$
\frac{\partial J}{\partial \mathbf{z}}
= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{h}}, \frac{\partial \mathbf{h}}{\partial \mathbf{z}}\right)
= \frac{\partial J}{\partial \mathbf{h}} \odot \phi'\left(\mathbf{z}\right).
$$

Finally, we can obtain the gradient
$\partial J/\partial \mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$
of the model parameters closest to the input layer.
According to the chain rule, we get

$$
\frac{\partial J}{\partial \mathbf{W}^{(1)}}
= \textrm{prod}\left(\frac{\partial J}{\partial \mathbf{z}}, \frac{\partial \mathbf{z}}{\partial \mathbf{W}^{(1)}}\right) + \textrm{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf{W}^{(1)}}\right)
= \frac{\partial J}{\partial \mathbf{z}} \mathbf{x}^\top + \lambda \mathbf{W}^{(1)}.
$$



## Training Neural Networks

When training neural networks,
forward and backward propagation depend on each other.
In particular, for forward propagation,
we traverse the computational graph in the direction of dependencies
and compute all the variables on its path.
These are then used for backpropagation
where the compute order on the graph is reversed.

Take the aforementioned simple network as an illustrative example.
On the one hand,
computing the regularization term :eqref:`eq_forward-s`
during forward propagation
depends on the current values of model parameters $\mathbf{W}^{(1)}$ and $\mathbf{W}^{(2)}$.
They are given by the optimization algorithm according to backpropagation in the most recent iteration.
On the other hand,
the gradient calculation for the parameter
:eqref:`eq_backprop-J-h` during backpropagation
depends on the current value of the hidden layer output $\mathbf{h}$,
which is given by forward propagation.


Therefore when training neural networks, once model parameters are initialized,
we alternate forward propagation with backpropagation,
updating model parameters using gradients given by backpropagation.
Note that backpropagation reuses the stored intermediate values from forward propagation to avoid duplicate calculations.
One of the consequences is that we need to retain
the intermediate values until backpropagation is complete.
This is also one of the reasons why training
requires significantly more memory than plain prediction.
Besides, the size of such intermediate values is roughly
proportional to the number of network layers and the batch size.
Thus,
training deeper networks using larger batch sizes
more easily leads to *out-of-memory* errors.


## Summary

Forward propagation sequentially calculates and stores intermediate variables within the computational graph defined by the neural network. It proceeds from the input to the output layer.
Backpropagation sequentially calculates and stores the gradients of intermediate variables and parameters within the neural network in the reversed order.
When training deep learning models, forward propagation and backpropagation are interdependent,
and training requires significantly more memory than prediction.


## Exercises

1. Assume that the inputs $\mathbf{X}$ to some scalar function $f$ are $n \times m$ matrices. What is the dimensionality of the gradient of $f$ with respect to $\mathbf{X}$?
1. Add a bias to the hidden layer of the model described in this section (you do not need to include bias in the regularization term).
    1. Draw the corresponding computational graph.
    1. Derive the forward and backward propagation equations.
1. Compute the memory footprint for training and prediction in the model described in this section.
1. Assume that you want to compute second derivatives. What happens to the computational graph? How long do you expect the calculation to take?
1. Assume that the computational graph is too large for your GPU.
    1. Can you partition it over more than one GPU?
    1. What are the advantages and disadvantages over training on a smaller minibatch?

[Discussions](https://discuss.d2l.ai/t/102)


Forward propogation:

Assume that the input x is of R^d, and hidden layer doesn't have a bias (just for simplicity)

Then, our hidden layer weights W_1 is of R^h * d

So, our intermediate variable is z = W_1x

Then we apply our nonlinearity: h = ϕ(z)

Then, we have a vector h of R^h

We can then get our output, where the weights W_2 are of R^q * h

So, o = W_2h

Assume loss function = l, then we can calculate the loss for a single example: L = l(o, y)

As for l2 reg (discussed in more detail later), we have:

s = lambda/2 * (frobenius(W_1)^2 + frobenius(W_2)^2)

So, our regularized loss, also called the objective function, is J = L + s

Here is the computational graph of this network (squares denote variables and circles denote operators. The lower-left corner signifies the input and the upper-right corner is the output. Notice that the directions of the arrows, which illustrate data flow, are primarily rightward and upward):



Backpropagation:

Backprop is where we calculate the gradient of the neural network parameters. We do this in reverse, from the output to the input layer, using the chain rule.  Intermediate values (the partial derivatives) are stored during this process, assuming we are calculating the gradient with respect to the parameters.

If we have functions Z = g(Y) and Y = f(X), where the input and output are tensors of arbitrary shape, We can calulate the derivative of Z with respect to X as:

∂Z/∂X = prod(∂Z, ∂X)

prod multiplies its arguments, and takes into account transposition and swapping input position and that sort of stuff. This is simple for vectors, it is just a matrix matrix multiplication. Higher dimensional tensors have their corresponding operations. prod is used to hide this overhead.

We will use the same network as for forward pass above.

So, we are trying to find the gradients ∂J/∂W_1 and ∂J/∂W_2

We do this by starting at J and working our way back.

First step: calculate the gradients of the objective function, J = L + s

∂J/∂L = 1

∂J/∂s = 1


Seconds step: Calculate the gradients of the objective function with respect to the output layer o, using chain rule:

∂L/∂o = prod(∂J/∂L, ∂L/∂o) = ∂L/∂o, R^q


Third step: Caluclate the gradients of the regularization with wrt both parameters:

s = lambda/2 * (frobenius(W_1)^2 + frobenius(W_2)^2)

∂s/∂W_1 = lambda * W_1

∂s/∂W_2 = lambda * W_2


Fourth step: Calculate the gradients of the layer closest to the output, ∂J/∂W_2, R^q * h

o = W_2h

∂J/∂W_2 = prod(∂J/∂o, ∂o/∂W_2) + prod(∂J/∂s, ∂s/∂W_2)

∂J/∂W_2 = (∂J/∂o)hT + lambda * W_2


Fifth step: We then continue on this process, from the output layer to the hidden layer, calculate the gradient ∂J/∂h, R^h

o = W_2h

∂J/∂h = prod(∂J/∂o, ∂o/∂h) = W_2T(∂J/∂o)


Sixth step: As the activation function ϕ applies elementwise, to find the gradient of intermediate variable z, ∂J/∂z, R^h, we use the elementwise operator ⊙. (Also because it applies elementwise, this is why neurons that are not activated do not recieved updates when using ReLU)

h = ϕ(z)

∂J/∂z = prod(∂J/∂h, ∂h/∂z) = (∂J/∂h)⊙ϕ'(z)


Last step: Now, we can obtain ∂J/∂W_1, R^h * d, the gradients of the weights closest to the input layer

z = W_1x

∂J/∂W_1 = prod(∂J/∂z, ∂z/∂W_1) + prod(∂J/∂s, ∂s/∂W_1)

∂J/∂W_1 = (∂J/∂z)xT + lambda * W_1



Training Neural Networks:

Clearly, forward and backward propagation depend on eachother. During forward propagation, we traverse the computational graph, and compute all variables on it. In backwards propagation, we use the values of the variables in reverse while finding the gradients.

In the computational graph of our network above, this is clear. In our forward pass, we use the current values of W_1 and W_2 to find the regularization constant, and these weights themselves were given through the last backwards pass. And for calculating the gradients of W_2, we need the current value of h, which was given through the forward pass.

As such, after our model parameters are initialized, we switch between forward and backward passes, updating the parameters using the gradients from backpropagation. We store the intermediate values of each forward pass, as we use them again for the corresponding backward pass so that we do not have to calculate it twice. This is why training requires more memory than inference, and the size of intermediate values is fairly proportional to the number of layers and batch size. Due to storing many parameters, this leads to a higher risk of out of memory errors.



Exercises:

1.Assume that the inputs  X to some scalar function  f  are  n * m  matrices. What is the dimensionality of the gradient of  f  with respect to  X?

The dimensionality of the gradient wrt to the inputs is just the dimensionality of the inputs. As such, the gradients of f wrt X is n * m.

2. Add a bias to the hidden layer of the model described in this section (you do not need to include bias in the regularization term).

2.1 Draw the corresponding computational graph.


2.2 Derive the forward and backward propagation equations.

Forward:

Assume that the input x is of R^d, and hidden layer bias b_1 of R^h

Then, our hidden layer weights W_1 is of R^h * d

So, our intermediate variable is z = W_1x + b_1

Then we apply our nonlinearity: h = ϕ(z)

Then, we have a vector h of R^h

We can then get our output, where the weights W_2 are of R^q * h

So, o = W_2h

Assume loss function = l, then we can calculate the loss for a single example: L = l(o, y)

As for l2 reg (discussed in more detail later), we have:

s = lambda/2 * (frobenius(W_1)^2 + frobenius(W_2)^2)

So, our regularized loss, also called the objective function, is J = L + s




Backward:

First step: calculate the gradients of the objective function, J = L + s

∂J/∂L = 1

∂J/∂s = 1


Seconds step: Calculate the gradients of the objective function with respect to the output layer o, using chain rule:

∂L/∂o = prod(∂J/∂L, ∂L/∂o) = ∂L/∂o, R^q


Third step: Caluclate the gradients of the regularization with wrt both parameters:

s = lambda/2 * (frobenius(W_1)^2 + frobenius(W_2)^2)

∂s/∂W_1 = lambda * W_1

∂s/∂W_2 = lambda * W_2


Fourth step: Calculate the gradients of the layer closest to the output, ∂J/∂W_2, R^q * h

o = W_2h

∂J/∂W_2 = prod(∂J/∂o, ∂o/∂W_2) + prod(∂J/∂s, ∂s/∂W_2)

∂J/∂W_2 = (∂J/∂o)hT + lambda * W_2


Fifth step: We then continue on this process, from the output layer to the hidden layer, calculate the gradient ∂J/∂h, R^h

o = W_2h

∂J/∂h = prod(∂J/∂o, ∂o/∂h) = W_2T(∂J/∂o)


Sixth step: As the activation function ϕ applies elementwise, to find the gradient of intermediate variable z, ∂J/∂z, R^h, we use the elementwise operator ⊙. (Also because it applies elementwise, this is why neurons that are not activated do not recieved updates when using ReLU)

h = ϕ(z)

∂J/∂z = prod(∂J/∂h, ∂h/∂z) = (∂J/∂h)⊙ϕ'(z)


Sixth step: Now, we can obtain ∂J/∂W_1, R^h * d, the gradients of the weights closest to the input layer

z = W_1x + b_1

∂J/∂W_1 = prod(∂J/∂z, ∂z/∂W_1) + prod(∂J/∂s, ∂s/∂W_1)

∂J/∂W_1 = (∂J/∂z)xT + lambda * W_1


Seventh step: Now, we can obtain ∂J/∂W_1, R^h, the gradient of the bias

z = W_1x + b_1

∂J/∂b_1 = prod(∂J/∂z, ∂z/∂b_1)

∂J/∂b_1 = (∂J/∂z)


The derivative of the weights doesn't change, because these are affine functions and bias doesn't affect slope.



3. Compute the memory footprint for training and prediction in the model described in this section.

we used a 3 layer MLP, with dimensions R ^:

input = d

W_1 = d * h

b_1 = h

z = h

W_2 = h * q

output = q

During training, for the forward pass we store all of these, so we store (data type size) * (d + d * h + h + h + h * q + q) = (data type) * (d + d * h + 2 * h + h * q + q) bits as a rough estimate. And then for our backward pass, we use these to calculate the gradients. For example, during backpropagation, when we compute ∂J/∂W_2 = (∂J/∂o)hT + lambda * W_2, we need the value of h that was computed during the forward pass. If we had discarded h after computing o, we'd have to recompute it, which defeats the purpose of backpropagation's efficiency. Which means we need to store (d * h + h + h * q) more, and update W_1, W_2, and b_1. Then, we can clear everything other than those 3. This cycle then repeats.

During prediction, since we are no longer updating the model, we do not need to store these after they are used. As we progress from input to output, once we finish the step where the values are used in calculations we can clear them. Like after you are finished W_1x, you can clear x, which is input of R^d.

So, the difference is that you have to retain the intermediate values during training, but not during inference. This means that training requires vastly more memory, and is far more likely to be subject to out of memory errors, especially as models get deeper, since amount of intermediate values stored is roughly proportional to batch_size * depth * (neurons_per_layer + parameters_per_layer).


4. Assume that you want to compute second derivatives. What happens to the computational graph? How long do you expect the calculation to take?

The second derivative, or hessian, means you take the derivative of the gradient. This means that if your gradient is of R ^ n, the hessian will be of R ^ n * n. This doesn't affect the computational graph for the forward pass, as the hessian is based off of the gradient, which we already have when doing a backward pass. However, during the backward pass, we need to store the size of the square of each weight and bias as well now.

Additionally, the result of this is that our graph is no longer identical forward and backward, as after computing the gradient, you treat each component f_i(x) as a scalar function, and compute the derivative of all f_i wrt to that f_i. As such, you need n extra backward passes (using the naive approach), to get the hessian.

This should go from O(n) to O(n^2), which is obviously terrible, which is why in practice the full hessian is not calculated, and methods such as diagonal approximations (only diagonals of hessian), or low rank approximations are used.



5. Assume that the computational graph is too large for your GPU.

5.1 Can you partition it over more than one GPU?

Yes, you can.

There are 2 main strategies.

Pipeline parallelism:
If you take any variable on the computational graph, the backward pass path to whatever foundational inputs lead to that variable are all the variables you need to store. However, each variable after that only needs access to the last "step" of variables.

So in our example, where we have x -> z -> h -> o, if you store x, z, and h on GPU 1, and o on GPU 2, then for GPU 2 to calculate o = W_2h, it only needs h from GPU 1 and doesn't need to know x and z.

This reduces intermediate storage per gpu, but also means that there are dependencies, as each GPU must wait on the previous one.

Data/Model parallelism:
We can split inputs and weights and biases and all that across gpus, but keeping the full computational graph. Because dot products are elementwise independent, each gpu can store a subset of the tensors, and then each return a scalar which is summed to make the final scalar. Each GPU handles part of the computation and results get combined. This means that there is no waiting for other GPUs (well at least not for sequential resources in the same way), as they operate breadthwise parallel.


5.2

The advantages of a smaller minibatches is that intermediate values stores grow roughly proportional to batch_size * depth * (neurons_per_layer + parameters_per_layer). As such, a smaller batch size gives a linearly proportional reduction in the memory needed. This is obviously good, because it reduces the risk of out of memory errors.

The disadvantages is that minibatches have higher variance and are less representative of the underlying distribution, which is inherent to their smaller size. As such, if you use anything like batchnorm, it will be less accurate, which can affect how well it converges.

Additonally, GPU are designed for extreme parallelization. Using smaller minibatches just means you are not taking full advantage of that, as it means:
Lower parallelization (underutilized GPU cores)
Higher overhead per operation
Worse arithmetic intensity (ratio of compute to memory operations)

So in general, you want to maximize the minibatch size as much as your memory allows for.
